# Running germ selection on multiple processors
The code below should be put into a script and run using mpiexec.  It's primary function is to pass a MPI Comm object to `pygsti.algorithms.germselection.build_up_breadth`.

In [1]:
from __future__ import print_function
import time

import pygsti
import pygsti.construction as pc
from pygsti.construction import std2Q_XYICNOT
from pygsti.algorithms import germselection as germsel

from mpi4py import MPI
comm = MPI.COMM_WORLD

def do_greedy_germsel(gs_target, forced_germs, candidate_counts,
                      seedStart, outFilename, comm):
    #candidate_counts is a dict of keys = germ lengths, values = # of germs at that length                                                            

    tStart = time.time()

    candidate_germs = []
    for i,(germLength, count) in enumerate(candidate_counts.items()):
        if count == "all upto":
            candidate_germs.extend( pc.list_all_gatestrings_without_powers_and_cycles(
                    gs_target.gates.keys(), maxLength=germLength) )
        else:
            candidate_germs.extend( pc.list_random_gatestrings_onelen(
                    gs_target.gates.keys(), germLength, count, seed=seedStart+i))

    available_germs = pygsti.tools.remove_duplicates( forced_germs + candidate_germs )
    print("%d available germs" % len(available_germs))
    germs = germsel.build_up_breadth(gs_target, available_germs,
                     randomizationStrength=1e-3, numCopies=3, seed=1234,
                     gatePenalty=10.0, scoreFunc='all', tol=1e-6, threshold=1e5,
                     pretest=False, force=forced_germs, verbosity=5, comm=comm, memLimit=0.5*(1024**3))

    if comm is None or comm.Get_rank() == 0:
        print("Germs (%d) = \n" % len(germs), "\n".join(map(str,germs)))
        print("Total time = %gs" % (time.time()-tStart))
        pickle.dump(germs,open(outFilename,"wb"))
    return germs
                                                                                                                                         
#2Q case                                                                                                                                              
gs_target = std2Q_XYICNOT.gs_target
forced_germs = pygsti.construction.gatestring_list([(gl,) for gl in gs_target.gates.keys()]) #singletons                                                                                      
candidate_counts = { 3:"all upto", 4:30, 5:20, 6:20, 7:20, 8:20} # germLength:num_candidates                                                          
seedStart = 4
do_greedy_germsel(gs_target, forced_germs, candidate_counts,
                  seedStart, "germs_EXAMPLE.pkl", comm)

201 available germs
Starting germ set optimization. Lower score is better.
Memory estimate of 13.6 GB (0.5 GB limit) for all-Jac mode.
Memory estimate of 0.2 GB (0.5 GB limit) for single-Jac mode.
    Initial germ set computation Iter 1 of 6 Gii: 
    Initial germ set computation Iter 2 of 6 Gix: 
    Initial germ set computation Iter 3 of 6 Giy: 
    Initial germ set computation Iter 4 of 6 Gxi: 
    Initial germ set computation Iter 5 of 6 Gyi: 
    Initial germ set computation Iter 6 of 6 Gcnot: 
  Outer iteration: 1 of 1282 amplified, 6 germs
    Inner iter over candidate germs Iter 001 of 195 GiiGix: 
      Score: major=-114.0 minor=275.75502325607516, N: 194
    Inner iter over candidate germs Iter 002 of 195 GiiGiy: 
      Score: major=-114.0 minor=275.7522442827053, N: 194
    Inner iter over candidate germs Iter 003 of 195 GiiGxi: 
      Score: major=-114.0 minor=303.1298571245524, N: 194
    Inner iter over candidate germs Iter 004 of 195 GiiGyi: 
      Score: major=-114.0 mi

KeyboardInterrupt: 

Above is **keyboard-interrupted on purpose**, as this output was produced with a single processor and it would have taken a very long time.
